MUlti layer inference

In [ ]:
def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 4
    param_size_GB = param_size_bytes / (1024 ** 3)
    return int(device_memory/param_size_GB)-drop

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])

    def forward(self, x, cos, sin,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x
layers=model.model.layers


# Usage
# compiled_block = torch.compile(FusedBlock(layers[i:j]))

def compute(model, x, layers_per_chunk, cos, sin,k,max_new_tokens,attention_mask):
    total_layers = len(model.model.layers)
    if layers_per_chunk >= total_layers:
        # model = model.to('cuda')
        # out = model({'inputs_embeds': x})['logits'][:, -1, :]
        # return out.argmax(-1)
        i=0
        l=total_layers
        m=Module(model.model.layers,i,i+l).to('cuda')
        x=m(x.to('cuda'),cos.to('cuda'),sin.to('cuda'),attention_mask.to('cuda'))
        if(k==max_new_tokens-1):
            m=m.to('cpu')
            torch.cuda.empty_cache()
        return x
    i = 0
    rem = total_layers
    while rem > 0:
        l = min(layers_per_chunk, rem)
        m = Module(model.model.layers, i, i + l).to('cuda')
        # m=torch.compile(m)
        # Run chunk
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'),attention_mask)
        # if(k==max_new_tokens-1):
        #     m=m.to('cpu')
        #     torch.cuda.empty_cache()
        m=m.to('cpu')
        torch.cuda.empty_cache()
        # update indices
        i += l
        rem -= l

    return x

def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu')
    input_ids=prompt['input_ids'].to('cuda')
    attention_mask=prompt['attention_mask']
    del prompt
    for i in range(tokens):
        torch.cuda.empty_cache()
        model.model.embed_tokens.to('cuda')
        x = model.model.embed_tokens(input_ids)
        # input_ids.to('cpu')
        # torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cpu')
        # torch.cuda.empty_cache()
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        rotary_emb = model.model.rotary_emb
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        cnt=0
        with torch.no_grad():
            l1=get_layers(model)
            x=compute(model,x,15,cos,sin,i,tokens,attention_mask)
            model.model.norm.to('cuda')
            x=model.model.norm(x)
            # model.model.norm.to('cpu')
            # torch.cuda.empty_cache()
            model.lm_head.to('cuda')
            x=model.lm_head(x)
            # model.lm_head.to('cpu')
            # torch.cuda.empty_cache()
        
        x1=x[:,-1,:]
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        print(tok.decode(x1[0]),end="")
        input_ids=torch.cat([input_ids,x1],dim=-1) 
        # x1=x.argmax(-1)
        # print(tok.decode(x1[0]),end="\n")
        # question=question+" "+tok.decode(x1[0])
        

Single layer infernece

In [ ]:
def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu')
    input_ids=prompt['input_ids'].to('cuda')
    attention_mask=prompt['attention_mask']
    del prompt
    for i in range(tokens):
        torch.cuda.empty_cache()
        model.model.embed_tokens.to('cuda')
        x = model.model.embed_tokens(input_ids)
        input_ids.to('cpu')
        # del input_ids
        torch.cuda.empty_cache()
        model.model.embed_tokens.to('cpu')
        torch.cuda.empty_cache()
        # Safe check
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        rotary_emb = model.model.rotary_emb
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        cnt=0
        for i in model.model.layers:
            cnt+=1
            # print(cnt)
            i.to('cuda')
            x.to('cuda')
            # print(x)
            with torch.no_grad():
                x=i(x,position_embeddings=(cos,sin))
                # print(x)
            i.to('cpu')
            torch.cuda.empty_cache()
            x=x[0]
            # x.to('cpu')
            torch.cuda.empty_cache()
            
        with torch.no_grad():
            model.model.norm.to('cuda')
            x=model.model.norm(x)
            model.model.norm.to('cpu')
            torch.cuda.empty_cache()
            model.lm_head.to('cuda')
            x=model.lm_head(x)
            model.lm_head.to('cpu')
            torch.cuda.empty_cache()
        
        x1=x[:,-1,:]
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        print(tok.decode(x1[0]),end="")
        input_ids=torch.cat([input_ids,x1],dim=-1) 
        # x1=x.argmax(-1)
        # print(tok.decode(x1[0]),end="\n")
        # question=question+" "+tok.decode(x1[0])
        